In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import svm
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.linear_model import Lasso,LinearRegression,LogisticRegression,ElasticNet
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier,BaseDecisionTree,DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier,StackingClassifier,RandomForestClassifier,RandomForestRegressor,GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder

import warnings
# warnings.filterwarnings("ignore")

le = LabelEncoder()
ohe = OneHotEncoder()
sc = StandardScaler()

rfc =  RandomForestClassifier(n_estimators=100)
rfr = RandomForestRegressor(n_estimators=100)
lr = LinearRegression()
lgr = LogisticRegression()
gnb = GaussianNB()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier()
svrc = svm.SVR()
lasso = Lasso(alpha=0.1)
bagc = BaggingClassifier()

models = [rfc,lr,lgr,gnb,mnb,dtc,svrc,lasso,bagc]

In [2]:
data = pd.read_csv('train.csv')
data

,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,total_amount,pickup_location_id,dropoff_location_id,year,month,day,day_of_week,hour_of_day,trip_duration,calculated_total_amount
0,9.01,1,N,1,26.0,0.0,0.5,8.14,5.76,0.3,40.70,262,138,2018,3,7,2,6,2131.0,24.30
1,0.20,1,N,1,3.0,0.0,0.5,0.75,0.00,0.3,4.55,263,236,2018,2,25,6,10,2377.0,37.40
2,9.65,1,N,1,41.5,0.0,0.5,9.61,5.76,0.3,57.67,138,230,2018,1,29,0,8,1286.0,30.36
3,9.50,1,N,1,30.0,0.5,0.5,9.25,5.76,0.3,46.31,186,138,2018,9,25,1,20,2586.0,4.30
4,5.80,1,N,1,21.5,0.5,0.5,4.56,0.00,0.3,27.36,162,87,2018,8,20,0,21,1575.0,23.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,22.43,1,N,1,59.5,0.5,0.5,10.00,5.76,0.3,76.56,132,259,2018,5,23,2,23,2998.0,29.76
34996,9.16,1,N,1,30.0,0.0,0.5,6.58,5.76,0.3,43.14,24,138,2018,4,4,2,9,1419.0,29.15
34997,6.78,1,N,1,23.0,0.0,0.5,5.95,0.00,0.3,29.75,140,231,2018,3,7,2,8,2459.0,42.67
34998,0.26,1,N,2,3.0,0.0,0.5,0.00,0.00,0.3,3.80,142,142,2018,4,18,2,8,1932.0,26.73


In [3]:
data.isna().sum()

trip_distance              0
rate_code                  0
store_and_fwd_flag         0
payment_type               0
fare_amount                0
extra                      0
mta_tax                    0
tip_amount                 0
tolls_amount               0
imp_surcharge              0
total_amount               0
pickup_location_id         0
dropoff_location_id        0
year                       0
month                      0
day                        0
day_of_week                0
hour_of_day                0
trip_duration              0
calculated_total_amount    0
dtype: int64

In [4]:
data.drop(['year','month','day','day_of_week',"extra",'mta_tax','tip_amount','tolls_amount','imp_surcharge','pickup_location_id','dropoff_location_id'],axis='columns',inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   trip_distance            35000 non-null  float64
 1   rate_code                35000 non-null  int64  
 2   store_and_fwd_flag       35000 non-null  object 
 3   payment_type             35000 non-null  int64  
 4   fare_amount              35000 non-null  float64
 5   total_amount             35000 non-null  float64
 6   hour_of_day              35000 non-null  int64  
 7   trip_duration            35000 non-null  float64
 8   calculated_total_amount  35000 non-null  float64
dtypes: float64(5), int64(3), object(1)
memory usage: 2.4+ MB


In [5]:
data.nunique()

trip_distance              2317
rate_code                     5
store_and_fwd_flag            2
payment_type                  4
fare_amount                 360
total_amount               2692
hour_of_day                  24
trip_duration              3913
calculated_total_amount    2678
dtype: int64

<img src='heatmap.png'>
<!-- # temp = data.corr()
# sns.heatmap(temp) -->

<img src='tripduration.png'>
<!-- sns.displot(data['trip_duration'],bins=50) -->

In [6]:
data['trip_duration'].describe()

count    35000.00000
mean      2257.38360
std       5112.41526
min          1.00000
25%       1448.00000
50%       1862.00000
75%       2340.00000
max      86347.00000
Name: trip_duration, dtype: float64

In [7]:
high_trip = data[data['trip_duration'] > 6000]
print(len(high_trip))
high_trip.head()

198


,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration,calculated_total_amount
174,8.48,1,N,1,24.5,37.87,23,86248.0,30.35
243,9.18,1,N,2,28.5,35.56,21,7185.0,32.80
272,26.80,1,N,1,71.5,83.30,14,6034.0,31.62
308,7.77,1,N,1,22.5,36.95,23,86033.0,43.87
369,8.88,1,N,1,28.0,30.38,1,84971.0,41.06


In [8]:
df1 = data[~(data['trip_duration'] < 100)]
print(len(df1))
df1.head()

33866


,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration,calculated_total_amount
0,9.01,1,N,1,26.0,40.70,6,2131.0,24.30
1,0.20,1,N,1,3.0,4.55,10,2377.0,37.40
2,9.65,1,N,1,41.5,57.67,8,1286.0,30.36
3,9.50,1,N,1,30.0,46.31,20,2586.0,4.30
4,5.80,1,N,1,21.5,27.36,21,1575.0,23.80


In [9]:
data['trip_distance'].describe()

count    35000.000000
mean         9.088815
std          4.496854
min          0.010000
25%          6.470000
50%          8.700000
75%         10.990000
max         79.010000
Name: trip_distance, dtype: float64

In [10]:
print(len(df1))
df2 = df1[~(df1['trip_distance'] < 0.1)]
print(len(df2))
df2.head()

33866
33697


,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration,calculated_total_amount
0,9.01,1,N,1,26.0,40.70,6,2131.0,24.30
1,0.20,1,N,1,3.0,4.55,10,2377.0,37.40
2,9.65,1,N,1,41.5,57.67,8,1286.0,30.36
3,9.50,1,N,1,30.0,46.31,20,2586.0,4.30
4,5.80,1,N,1,21.5,27.36,21,1575.0,23.80


In [11]:
data.rate_code.value_counts()

1    33664
3      650
5      501
4      182
2        3
Name: rate_code, dtype: int64

In [12]:
data.store_and_fwd_flag.value_counts()

N    34823
Y      177
Name: store_and_fwd_flag, dtype: int64

In [13]:
data.payment_type.value_counts()

1    30899
2     3926
3      132
4       43
Name: payment_type, dtype: int64

<img src='calculated_total_amount.png'>

<img src='fare_amt.png'>

In [14]:
data['calculated_total_amount'].describe()

count    35000.000000
mean        41.324249
std         19.167043
min          0.310000
25%         30.300000
50%         38.460000
75%         48.200000
max        468.350000
Name: calculated_total_amount, dtype: float64

In [15]:
df3 = df2[~(df2['calculated_total_amount'] >150)]
print(len(df3))
df3.head()

33586


,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration,calculated_total_amount
0,9.01,1,N,1,26.0,40.70,6,2131.0,24.30
1,0.20,1,N,1,3.0,4.55,10,2377.0,37.40
2,9.65,1,N,1,41.5,57.67,8,1286.0,30.36
3,9.50,1,N,1,30.0,46.31,20,2586.0,4.30
4,5.80,1,N,1,21.5,27.36,21,1575.0,23.80


In [16]:
df4 = df3[~(df3['calculated_total_amount'] < 10)]
print(len(df4))

32520


In [17]:
df5 = df4[~(df4['fare_amount'] > (df4['calculated_total_amount']*5))]
print(len(df5))
df5.head()

32491


,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration,calculated_total_amount
0,9.01,1,N,1,26.0,40.70,6,2131.0,24.30
1,0.20,1,N,1,3.0,4.55,10,2377.0,37.40
2,9.65,1,N,1,41.5,57.67,8,1286.0,30.36
4,5.80,1,N,1,21.5,27.36,21,1575.0,23.80
5,12.65,1,N,1,41.5,48.08,7,2029.0,30.95


In [18]:
df6 = df5[~((df5['fare_amount']*5) < df5['calculated_total_amount'])]
print(len(df6))
df7 = df6[~(df6['trip_duration'] > 6000)]
df7.shape

31542


(31355, 9)

In [19]:
df7.store_and_fwd_flag = df7.store_and_fwd_flag.replace({'N':1,'Y':0})
# df7.trip_duration = df7.trip_duration.apply(lambda x: x/60)
df = df7.copy()
df7

C:\Users\Akshay L Telang\AppData\Local\Temp\ipykernel_9724\471546477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.store_and_fwd_flag = df7.store_and_fwd_flag.replace({'N':1,'Y':0})


,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration,calculated_total_amount
0,9.01,1,1,1,26.0,40.70,6,2131.0,24.30
2,9.65,1,1,1,41.5,57.67,8,1286.0,30.36
4,5.80,1,1,1,21.5,27.36,21,1575.0,23.80
5,12.65,1,1,1,41.5,48.08,7,2029.0,30.95
6,8.92,1,1,1,27.0,40.27,10,2144.0,46.86
...,...,...,...,...,...,...,...,...,...
34994,4.91,1,1,1,22.5,30.38,16,1869.0,56.00
34995,22.43,1,1,1,59.5,76.56,23,2998.0,29.76
34996,9.16,1,1,1,30.0,43.14,9,1419.0,29.15
34997,6.78,1,1,1,23.0,29.75,8,2459.0,42.67


In [20]:
df.describe()

,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration,calculated_total_amount
count,31355.000000,31355.000000,31355.000000,31355.000000,31355.000000,31355.000000,31355.000000,31355.000000,31355.000000
mean,9.353849,1.107575,0.995280,1.108085,32.716195,42.502041,13.831032,1991.118195,42.034923
std,4.130501,0.571097,0.068542,0.327100,12.771519,17.120495,6.205088,716.361246,15.188131
min,0.100000,1.000000,0.000000,1.000000,8.030000,10.150000,0.000000,100.000000,10.060000
25%,6.700000,1.000000,1.000000,1.000000,24.500000,30.950000,10.000000,1488.000000,30.950000
50%,8.870000,1.000000,1.000000,1.000000,29.500000,39.070000,14.000000,1885.000000,39.070000
75%,11.080000,1.000000,1.000000,1.000000,36.500000,48.670000,19.000000,2349.000000,48.670000
max,73.480000,5.000000,1.000000,4.000000,240.000000,315.380000,23.000000,5965.000000,149.950000


In [21]:
x = df.drop('calculated_total_amount',axis='columns')
y = df.calculated_total_amount
x.head()

,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,total_amount,hour_of_day,trip_duration
0,9.01,1,1,1,26.0,40.70,6,2131.0
2,9.65,1,1,1,41.5,57.67,8,1286.0
4,5.80,1,1,1,21.5,27.36,21,1575.0
5,12.65,1,1,1,41.5,48.08,7,2029.0
6,8.92,1,1,1,27.0,40.27,10,2144.0


In [40]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15)
print(len(x_train))
print(len(y_test))

26651
4704


In [41]:
y_test.head()

1542     48.67
31446    80.15
29289    32.16
29633    27.80
29429    37.56
Name: calculated_total_amount, dtype: float64

In [24]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [42]:
pip = make_pipeline(StandardScaler(),KNeighborsRegressor())
pip.fit(x_train,y_train)
print(pip.score(x_test,y_test))
pip.predict(x_test[:5])

# 0.0009753094949324081
# array([41.79389053])

# -0.0002600116460729218
# array([42.20421902, 42.03557182, 42.00397139, 42.36373474, 42.07816884])


-0.19059741012139741


array([35.928, 34.326, 42.504, 34.838, 34.398])

In [43]:
KNN = KNeighborsRegressor()

KNN.fit(x_train,y_train)
print(KNN.score(x_test,y_test))
KNN.predict(x_test[:5])

# -0.1442943181950278
# array([41.122])


# -0.1792003275845071
# array([39.882, 54.146, 36.406, 33.646, 43.198])
#       42.44   51.36    46.88   27.36    42.07



# 17533    79.27
# 18432    40.56
# 2314     50.30
# 32628    54.45
# 31597    25.64

-0.19446784251628002


array([38.188, 39.61 , 32.03 , 46.748, 48.58 ])

In [27]:
rfr.fit(x,y)
print(rfr.score(x_test,y_test))
rfr.predict(x_test[:5])

# -0.0648554228315017
# array([44.2171, 50.8249, 44.8787, 47.019 , 45.8209])
#        42.44   51.36    46.88   27.36    42.07

# 17533    79.27
# 18432    40.56
# 2314     50.30
# 32628    54.45
# 31597    25.64

0.8491059197774535


array([48.8277, 40.352 , 41.7717, 38.1029, 73.435 ])

In [28]:
rfr.score(x_train,y_train)

0.8507627271597451

In [39]:
ss

NameError: name 'ss' is not defined

In [29]:
lasso.fit(x,y)
print(lasso.score(x_test,y_test))
lasso.predict(x_test[:5])

#    0.0007386907360135142
#    array([42.17461288])

# -0.00039111404599712074
# array([42.11175136, 41.91929405, 41.88176423, 41.74875436, 41.82424286]

# 17533    79.27
# 18432    40.56
# 2314     50.30
# 32628    54.45
# 31597    25.64

0.0003640485755247447


array([41.93441048, 42.06726295, 42.27653863, 42.1056259 , 42.04560965])

In [30]:
gm = GradientBoostingRegressor(n_estimators=100)
gm.fit(x,y)
print(gm.score(x_test,y_test))
gm.predict(x_test[:5])

# 0.0007386907360135142
# array([42.17461288])

# -0.00149653446140241
# array([41.99708892, 42.40879783, 41.97128264, 41.84697767, 42.08122521])
#        42.44      51.36        46.88        27.36        42.07


# 17533    79.27
# 18432    40.56
# 2314     50.30
# 32628    54.45
# 31597    25.64

0.015802835716481


array([40.29048183, 41.77307582, 42.00280097, 42.08657134, 41.6109065 ])

In [45]:
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")

pip = make_pipeline(StandardScaler(),dummy_regr)
pip.fit(x_train,y_train)
print(pip.score(x_test,y_test))
pip.predict(x_test[:5])

# dummy_regr.fit(x,y)
# print(dummy_regr.score(x_test,y_test))
# dummy_regr.predict(x_test[:5])

# -0.0009880735925749118
# array([41.25919968])

# -0.0005175614427486153
# array([41.98282128, 41.98282128, 41.98282128, 41.98282128, 41.98282128])
#        42.44      51.36        46.88        27.36        42.07

# 17533    79.27
# 18432    40.56
# 2314     50.30
# 32628    54.45
# 31597    25.64

-1.8719871507810737e-05


array([42.04492664, 42.04492664, 42.04492664, 42.04492664, 42.04492664])

In [32]:
from sklearn.ensemble import VotingRegressor 

r1 = KNeighborsRegressor()

lgr = VotingRegressor([('knn', r1), ('rfr', rfr)])
lgr.fit(x,y)
print(lgr.score(x_test,y_test))
lgr.predict(x_test[:5])

# 17668    45.67
# 14939    46.30
# 2206     35.80
# 31146    50.47
# 34770    48.67

# 17533    79.27
# 18432    40.56
# 2314     50.30
# 32628    54.45
# 31597    25.64

0.6124805696597951


array([44.38385, 37.72995, 41.5366 , 46.14785, 66.801  ])

In [33]:
test = pd.read_csv('test.csv')
test

,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,total_amount,pickup_location_id,dropoff_location_id,year,month,day,day_of_week,hour_of_day,trip_duration
0,15.70,1,N,2,45.0,0.0,0.5,0.00,0.00,0.3,45.80,132,146,2018,10,20,5,16,3967.0
1,7.30,1,N,1,25.5,0.0,0.5,7.89,0.00,0.3,34.19,186,69,2018,2,24,5,15,1491.0
2,14.87,1,N,1,40.5,0.5,0.5,8.36,0.00,0.3,50.16,138,89,2018,9,11,1,22,1395.0
3,2.90,1,N,2,22.0,0.0,0.5,0.00,0.00,0.3,22.80,90,162,2018,6,7,3,11,1742.0
4,8.96,1,N,1,32.0,0.0,0.5,0.00,5.76,0.3,38.56,138,186,2018,4,20,4,10,1623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,5.80,1,N,1,22.0,0.5,0.5,3.50,0.00,0.3,28.75,148,143,2018,5,5,5,0,1121.0
14996,10.50,1,N,1,35.5,0.5,0.5,7.35,0.00,0.3,44.15,163,85,2018,7,5,3,23,1744.0
14997,9.10,1,N,1,27.0,0.5,0.5,6.80,5.76,0.3,40.86,138,107,2018,9,13,3,20,1362.0
14998,9.50,1,N,1,29.5,0.0,0.5,7.00,5.76,0.3,43.06,170,138,2018,8,22,2,12,101.0


In [34]:
test.drop(['year','month','day','day_of_week',"extra",'mta_tax','tip_amount','tolls_amount','imp_surcharge','pickup_location_id','dropoff_location_id'],axis='columns',inplace=True)
test.store_and_fwd_flag = test.store_and_fwd_flag.replace({'N':1,'Y':0})
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   trip_distance       15000 non-null  float64
 1   rate_code           15000 non-null  int64  
 2   store_and_fwd_flag  15000 non-null  int64  
 3   payment_type        15000 non-null  int64  
 4   fare_amount         15000 non-null  float64
 5   total_amount        15000 non-null  float64
 6   hour_of_day         15000 non-null  int64  
 7   trip_duration       15000 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 937.6 KB


In [35]:
predicted = {'calculated_total_amount': KNN.predict(test)}
pre = pd.DataFrame(predicted)
pre.describe()

,calculated_total_amount
count,15000.000000
mean,41.909091
std,6.693412
min,23.464000
25%,37.262000
50%,40.819000
75%,45.546000
max,84.260000


In [36]:
#pre.to_csv('submmision_knn.csv',index=False)

In [37]:
predicted = {'calculated_total_amount': rfr.predict(test)}
pre = pd.DataFrame(predicted)
# pre.to_csv('submmision_rfr.csv',index=False)
pre.describe()

,calculated_total_amount
count,15000.000000
mean,43.003694
std,4.267114
min,30.440300
25%,40.119925
50%,42.536150
75%,45.367575
max,73.878000


In [38]:
predicted = {'calculated_total_amount': lgr.predict(test)}
pre = pd.DataFrame(predicted)
# pre.to_csv('submmision_voting.csv',index=False)
pre.describe()

,calculated_total_amount
count,15000.000000
mean,42.517182
std,4.389324
min,30.232400
25%,39.452587
50%,41.991575
75%,44.969988
max,70.844750
